In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import multiple_planets_gas_acc as code
import functions_pebble_accretion as peb
import numpy as np
from scipy import integrate
from functions import *
from functions_pebble_accretion import *
import functions_plotting as plot
from matplotlib import cm, ticker
from matplotlib import colors
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from tqdm.notebook import tqdm
import matplotlib.gridspec as gridspec
import cProfile 
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator
import os
import sim_loader as sim_load


color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code.sim_params.nr_planets))


In [ ]:
fig, axs = plt.subplots(1,1, figsize = (6,6))
t = np.linspace(0.1, 5)
params = code.Params()
axs.loglog(t, M_dot_star_t(t), color = "green", label = "Hartmann et al. 2016")
axs.loglog(t, M_dot_star_t_Mstar(t, params), color = "green", label = "Liu et al. 2019")

In [ ]:
params_dict = {'St_const': None, 
                'M_dot_star': None,
                #'M_dot_star': (1e-8*const.M_sun.cgs/u.yr).to(u.g/u.s),
                'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                'gas_accretion': False
                }
t_initial = 0.1
a_p0 = np.geomspace(50, 1e-1, num = 50)
t0= ([t_initial] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = (t_initial) #is needed to start the simulation at the right time?
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in
                }

epsilon_el = [1, 1e-2, 1e-2]
epsilon_h = [1, 0.5, 0.5]

params_lam_ext = code.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=epsilon_el[0], epsilon_heat=epsilon_h[0])
params_lam = code.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=epsilon_el[1], epsilon_heat=epsilon_h[1])
params_irr = code.Params(**params_dict, H_r_model='irradiated', epsilon_el=epsilon_el[2], epsilon_heat=epsilon_h[2])
peb_acc = code.PebbleAccretion(simplified_acc=False)
gas_acc = code.GasAccretion()

m0_lam_ext = M0_pla(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_lam_ext), params_lam_ext)
sim_params_lam_ext = code.SimulationParams(**sim_params_dict, m0=m0_lam_ext)
m0_lam = M0_pla(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_lam), params_lam)
sim_params_lam = code.SimulationParams(**sim_params_dict, m0=m0_lam)
m0_irr = M0_pla(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_irr), params_irr)
sim_params_irr = code.SimulationParams(**sim_params_dict, m0=m0_irr)

sim_lam_ext = code.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc,gas_acc = gas_acc,  params=params_lam_ext, sim_params=sim_params_lam_ext)
sim_lam = code.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc, gas_acc = gas_acc, params=params_lam, sim_params=sim_params_lam)
sim_irr = code.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc, gas_acc = gas_acc, params=params_irr, sim_params=sim_params_irr)


In [ ]:
sim_lam = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/models_5Myr/simulation_Lambrechts_mixed_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json")
sim_lam_ext = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/models_5Myr/simulation_Lambrechts_mixed_e_el_1_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json")
sim_irr = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/models_5Myr/simulation_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json")
sim_params_lam = sim_load.load_sim_params('sims/gas_acc/accretion_maps/models_5Myr/sim_params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
sim_params_lam_ext = sim_load.load_sim_params('sims/gas_acc/accretion_maps/models_5Myr/sim_params_Lambrechts_mixed_e_el_1_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
sim_params_irr = sim_load.load_sim_params('sims/gas_acc/accretion_maps/models_5Myr/sim_params_Lambrechts_mixed_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
params_lam = sim_load.load_params('sims/gas_acc/accretion_maps/models_5Myr/params_Lambrechts_mixed_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
params_lam_ext = sim_load.load_params('sims/gas_acc/accretion_maps/models_5Myr/params_Lambrechts_mixed_e_el_1_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
params_irr = sim_load.load_params('sims/gas_acc/accretion_maps/models_5Myr/params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')


labels = ['Irradiated', 'Surface heating', 'Midplane heating']
fig, axs = plt.subplots(1,3, figsize = (23,8))
num = 10000
clrs = ['grey','black']

# binning the mass on the y-axis between the min and max of the starting and final masses
m_min_lam = M0_pla(sim_lam.position[:,0].value, sim_params_lam.t_in, sigma_gas_steady_state(sim_lam.position[:,0].value, sim_params_lam.t_in, params_lam), params_lam).min()
m_min_lam_ext = M0_pla(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, sigma_gas_steady_state(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, params_lam_ext), params_lam_ext).min()
m_min_irr = M0_pla(sim_irr.position[:,0].value, sim_params_irr.t_in, sigma_gas_steady_state(sim_irr.position[:,0].value, sim_params_irr.t_in, params_irr), params_irr).min()

m_max_lam = M_peb_iso(sim_lam.position[:,0].value, sim_params_lam.t_in, params_lam).max()
m_max_lam_ext = M_peb_iso(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, params_lam_ext).max()
m_max_irr = M_peb_iso(sim_irr.position[:,0].value, sim_params_irr.t_in, params_irr).max()

M_in_min = min(m_min_lam, m_min_lam_ext, m_min_irr)
M_fin_max = max(m_max_lam,m_max_lam_ext,m_max_irr) 
m_bins = np.geomspace(M_in_min, M_fin_max, num = num)
#print("mass binning", m_bins.to(u.M_earth))
Z = [np.zeros((len(m_bins), sim_params_irr.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_irr.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_irr.nr_planets))*u.Myr]
Zmin = np.array([])*u.Myr
Zmax = np.array([])*u.Myr
for i,(sim, params, sim_params) in enumerate(zip([sim_irr, sim_lam, sim_lam_ext],[params_irr, params_lam, params_lam_ext], [sim_params_irr, sim_params_lam, sim_params_lam_ext])):

    for p in range(sim_params.nr_planets): #loop over the nr_planets
        for m in range(len(m_bins)): #loops over the mass of the planet (timesteps)
            t_idx = np.argmax(sim.mass[p].value >= m_bins[m])
            Z[i][m,p] = sim.time[t_idx].to(u.Myr)
    axs[i].loglog(sim.position[:,0], M_peb_iso(sim.position[:,0].value, sim_params.t_in, params), 
            label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at initial time, for the viscous case it decreases with time
    axs[i].loglog(sim.position[:,0], M_peb_iso(sim.position[:,0].value, sim_params.t_fin, params), 
            label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at final time

    axs[i].loglog(sim.position[:,0], M0_pla(sim.position[:,0].value, sim_params.t_in, 
                                                sigma_gas_steady_state(sim.position[:,0].value, sim_params.t_in, params), params), 
                                                label = "M_in", color = 'lightblue', linestyle = ':')
    axs[i].loglog(sim.position[:,0], M_3D_2DH_trans(sim.position[:,0].value, sim_params.t_in, params), color = 'black', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    axs[i].loglog(sim.position[:,0], M_3D_2DB_trans(sim.position[:,0].value, sim_params.t_fin, params), color = 'lightgrey', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    axs[i].loglog(sim.position[:,0], M_Bondi_Hill_trans(sim.position[:,0].value, sim_params.t_fin, params), color = 'green', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent

    R_irr_visc_in = plot.r_visc_irr(sim.position[:,0].value, sim_params.t_in, params, sim_params)
    R_irr_visc_fin = plot.r_visc_irr(sim.position[:,-1].value, sim_params.t_fin, params, sim_params)
    for j,r in enumerate([R_irr_visc_in, R_irr_visc_fin]):
        col = clrs[j]
        if r is not None:
            print("transition radii"+str(params.H_r_model),r)
            axs[i].axvline(r, ymin = 0, ymax = 0.06, color = col,  alpha = 0.8, linewidth = 2)
            arrow_dx = r/2.5
            axs[i].annotate("", xy=(r-arrow_dx, 2e-6), xytext=(r, 2e-6), 
                         size="large",horizontalalignment="center", verticalalignment="center",
                         arrowprops = dict(arrowstyle = '->', color=col,alpha = 0.6, lw = 2))

    
    # Filling of the pebble isolation mass
    axs[i].fill_between(sim.position[:,0].to(u.au).value, M_peb_iso(sim.position[:,0].value, sim_params.t_in, params),
                         M_peb_iso(sim.position[:,0].value, sim_params.t_fin, params),  color='slateblue',alpha=0.3)
    Zmin = np.append(Zmin, Z[i].min())
    Zmax = np.append(Zmax, Z[i].max())

levs = np.geomspace(sim_params.t_in,sim_params.t_fin, num = 20)
vmin = sim_params_irr.t_in
vmax = sim_params_irr.t_fin
print(vmin, vmax)
for i,(sim, params, sim_params) in enumerate(zip([sim_irr, sim_lam, sim_lam_ext],[params_irr, params_lam, params_lam_ext], [sim_params_irr, sim_params_lam, sim_params_lam_ext])):

    X,Y = np.meshgrid(sim.position[:,0], m_bins)

    #plot white for the regions where the planet is dead or non existent
    Z[i][Z[i].value<=1e-1] = np.nan
    cmap = mpl.colormaps["inferno"].reversed()
    cmap.set_bad(color='white')
    masked_data = np.ma.masked_invalid(Z[i].value)
    print(masked_data)
    CS = axs[i].contourf(X, Y, masked_data, levels= levs, norm=colors.LogNorm(vmin = vmin, vmax = vmax), boundaries=levs, cmap = cmap)

#handling the colorbar	
#fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(CS, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0,2.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('accretion time [Myr]', fontsize=25, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 18, labelsize = 18)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 12)


for i in range(3):
    # Set the title and axis labels
    axs[i].set_title(labels[i], fontsize = 25)
    axs[i].set_ylim(M_in_min, M_fin_max)
    axs[i].set_xlabel('r [AU]', fontsize = 25, labelpad=20)
    axs[i].tick_params(axis='both', which='major', size=15)
    axs[i].tick_params(axis='both', which='minor', size=10)
    axs[i].tick_params(axis='both', which='major', labelsize=20)
    # axs[i].figure.axes[-1].yaxis.label.set_size(25)
    # axs[i].figure.axes[-1].tick_params(axis='both', which='minor', labelsize=10)
    # axs[i].figure.axes[-1].tick_params(axis='both', which='major', labelsize=15)
    axs[i].set_ylim(1e-6, 1e2)
    plot.all_y_ticks(axs[i], 100)


#panel 1
axs[0].text(sim.position[30,0].to(u.au).value, 1.5e-4, 'initial mass', fontsize=18, color='lightblue', rotation=36, rotation_mode='anchor', ha='left', va='bottom')
axs[0].text(sim.position[30,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')
#panel 2
axs[1].text(sim.position[30,0].to(u.au).value, 5e-5, 'initial mass', fontsize=18, color='lightblue', rotation=40.5, rotation_mode='anchor', ha='left', va='bottom')
axs[1].text(sim.position[30,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')
#panel 3
axs[2].text(sim.position[30,0].to(u.au).value, 3e-5, 'initial mass', fontsize=18, color='lightblue', rotation=40.5, rotation_mode='anchor', ha='left', va='bottom')
axs[2].text(sim.position[28,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/heatmaps/models_vfrag1ms_pebiso3", bbox_inches='tight')


In [ ]:
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 10
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }
par = code.Params(**params_dict, H_r_model='irradiated', epsilon_el=1, epsilon_heat=1)
sim_par= code.SimulationParams(**sim_params_dict, m0=M0_pla(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, par), par))
print(params)
print(par)

# print(M_3D_2DH_trans(sim.position[:,0].value, sim_params.t_in, params))
fig, axs =plt.subplots(1,1, figsize = (8,8))
axs.loglog(sim.position[:,0].to(u.au), M_3D_2DH_trans(sim.position[:,0].value, sim_params.t_fin, params))
axs.set_ylim(1e-6, 1e2)
plot.all_y_ticks(axs, 100)

In [ ]:
labels = ['irradiated disc, $v_{\mathrm {frag}} = 1$ m/s', 'irradiated disc, $v_{\mathrm {frag}} = 10$ m/s']
fig, axs = plt.subplots(1,2, figsize = (12,6) )
num = 10000
clrs = ['grey','black']

sim_irr_1 = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/irr_vfrag/simulation_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json")
sim_irr_10 = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/irr_vfrag/simulation_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json")
sim_params_irr_1 = sim_load.load_sim_params('sims/gas_acc/accretion_maps/irr_vfrag/sim_params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
sim_params_irr_10 = sim_load.load_sim_params('sims/gas_acc/accretion_maps/irr_vfrag/sim_params_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json')
params_irr_1 = sim_load.load_params('sims/gas_acc/accretion_maps/irr_vfrag/params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.1_N_steps10000.json')
params_irr_10 = sim_load.load_params('sims/gas_acc/accretion_maps/irr_vfrag/params_irradiated_e_el_0.01_vfrag_10.0_planets_50_t0_0.1_N_steps10000.json')



# binning the mass on the y-axis between the min and max of the starting and final masses
m_min_lam_1 = M0_pla(sim_irr_1.position[:,0].value, sim_params_irr_1.t_in, sigma_gas_steady_state(sim_irr_1.position[:,0].value, sim_params_irr_1.t_in, params_irr_1), params_irr_1).min()
m_min_lam_10 = M0_pla(sim_irr_10.position[:,0].value, sim_params_irr_10.t_in, sigma_gas_steady_state(sim_irr_10.position[:,0].value, sim_params_irr_10.t_in, params_irr_10), params_irr_10).min()

m_max_lam_1 = M_peb_iso(sim_irr_1.position[:,0].value, sim_params_irr_1.t_in, params_irr_1).max()
m_max_lam_10 = M_peb_iso(sim_irr_10.position[:,0].value, sim_params_irr_10.t_in, params_irr_10).max()
M_in_min = m_min_lam_1
M_fin_max = m_max_lam_1

M_in_min = min(m_min_lam_1, m_min_lam_10)
M_fin_max = max(m_max_lam_1,m_max_lam_10) 
m_bins = np.geomspace(M_in_min, M_fin_max, num = num)
#print("mass binning", m_bins.to(u.M_earth))
Z = [np.zeros((len(m_bins), sim_params_irr_1.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_irr_1.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_irr_1.nr_planets))*u.Myr]
Zmin = np.array([])*u.Myr
Zmax = np.array([])*u.Myr
for i,(sim, params, sim_params) in enumerate(zip([sim_irr_1, sim_irr_10],[params_irr_1, params_irr_10], [sim_params_irr_1, sim_params_irr_10])):
    print(sim_params.t_in)
    for p in range(sim_params.nr_planets): #loop over the nr_planets
        for m in range(len(m_bins)): #loops over the mass of the planet (timesteps)
            t_idx = np.argmax(sim.mass[p].value >= m_bins[m])
            Z[i][m,p] = sim.time[t_idx].to(u.Myr)
    #axs[i].loglog(sim.position[:,0].to(u.au), M_peb_iso(sim.position[:,0], sim_params.t_in, params).to(u.M_earth), 
            #label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at initial time, for the viscous case it decreases with time
    axs[i].loglog(sim.position[:,0], M_peb_iso(sim.position[:,0].value, sim_params.t_fin, params), 
            label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at final time

    axs[i].loglog(sim.position[:,0], M0_pla(sim.position[:,0].value, sim_params.t_in, 
                                                sigma_gas_steady_state(sim.position[:,0].value, sim_params.t_in, params), params), 
                                                label = "M_in", color = 'lightblue', linestyle = ':')
    axs[i].loglog(sim.position[:,0], M_3D_2DB_trans(sim.position[:,0].value, sim_params.t_fin, params), color = 'lightgrey', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    axs[i].loglog(sim.position[:,0], M_3D_2DH_trans(sim.position[:,0].value, sim_params.t_fin, params), color = 'black', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    axs[i].loglog(sim.position[:,0], M_Bondi_Hill_trans(sim.position[:,0].value, sim_params.t_fin, params), color = 'green', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent

    Zmin = np.append(Zmin, Z[i].min())
    Zmax = np.append(Zmax, Z[i].max())

#levs = np.geomspace(Zmin.min(), Zmax.max())
levs = np.geomspace(sim_params.t_in,sim_params.t_fin, num = 20)

vmin = Zmin.min().value
vmax = Zmax.max().value
for i,(sim, params, sim_params) in enumerate(zip([sim_irr_1, sim_irr_10],[params_irr_1, params_irr_10], [sim_params_irr_1, sim_params_irr_10])):

    X,Y = np.meshgrid(sim.position[:,0], m_bins)

    #plot white for the regions where the planet is dead or non existent
    Z[i][Z[i].value<=1e-1] = np.nan
    cmap = mpl.colormaps["inferno"].reversed()
    cmap.set_bad(color='white')
    masked_data = np.ma.masked_invalid(Z[i].value)
    print(masked_data)
    CS = axs[i].contourf(X, Y, masked_data, levels= levs, norm=colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin), boundaries=levs, cmap = cmap)
    axs[i].fill_between(sim.position[:,0].value, M_peb_iso(sim.position[:,0], sim_params.t_in, params),
                        M_peb_iso(sim.position[:,0].value, sim_params.t_fin, params),  color='slateblue', alpha=0.2)


#handling the colorbar	
#fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar = fig.colorbar(CS, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs='auto'))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))

cbar.set_label('accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 13, labelsize = 13)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 9)


for i in range(2):
    # Set the title and axis labels
    axs[i].set_title(labels[i], fontsize = 20)
    axs[i].set_ylim(M_in_min, M_fin_max)
    axs[i].set_xlabel('r [AU]', fontsize = 25, labelpad=20)
    axs[i].tick_params(axis='both', which='major', size=15)
    axs[i].tick_params(axis='both', which='minor', size=10)
    axs[i].tick_params(axis='both', which='major', labelsize=15)
    axs[i].figure.axes[-1].yaxis.label.set_size(22)
    axs[i].figure.axes[-1].tick_params(axis='both', which='minor', labelsize=10)
    axs[i].figure.axes[-1].tick_params(axis='both', which='major', labelsize=15)
    axs[i].set_ylim(1e-6, 1e2)
    plot.all_y_ticks(axs[i], 100)

axs[0].text(sim.position[30,0].to(u.au).value, 1.5e-4, 'initial mass', fontsize=15, color='lightblue', rotation=36, rotation_mode='anchor', ha='left', va='bottom')
axs[0].text(sim.position[29,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=15, color='slateblue', rotation=17, rotation_mode='anchor', ha='left', va='bottom')
axs[0].text(sim.position[45,0].to(u.au).value, 8, '2D Hill', fontsize=15, color='black', rotation=-5, rotation_mode='anchor', ha='left', va='bottom')
axs[0].text(sim.position[45,0].to(u.au).value, 2, '3D', fontsize=15, color='black', rotation=-5, rotation_mode='anchor', ha='left', va='bottom')

#panel 2
axs[1].text(sim.position[30,0].to(u.au).value, 1.5e-4, 'initial mass', fontsize=15, color='lightblue', rotation=36, rotation_mode='anchor', ha='left', va='bottom')
axs[1].text(sim.position[29,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=15, color='slateblue', rotation=17, rotation_mode='anchor', ha='left', va='bottom')
axs[1].text(sim.position[15,0].to(u.au).value, 1e-1, '2D Hill', fontsize=15, color='black', rotation=29, rotation_mode='anchor', ha='left', va='bottom')
axs[1].text(sim.position[12,0].to(u.au).value, 3e-2, '3D', fontsize=15, color='black', rotation=29, rotation_mode='anchor', ha='left', va='bottom')
axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/heatmaps/irr_1_10m_s_Bondiline",  bbox_inches='tight')


In [ ]:
kepler_3_law_inverse (8).to(u.day)

In [ ]:
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 10
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }
par = code.Params(**params_dict, H_r_model='irradiated', epsilon_el=1, epsilon_heat=1)
def M_3D_2DH_trans_debug_drift_ep(position, t, params):

    stokes_peb = st_drift_epstein(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))

def M_3D_2DH_trans_debug_drift_st(position, t, params):

    stokes_peb = st_drift_stokes(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))
def M_3D_2DH_trans_debug_frag(position, t, params):

    stokes_peb = st_frag(position, t, params)

    return 3/4*(2*H_peb(stokes_peb, position, t, params)*(np.sqrt(2*np.pi))/np.pi)**3*(omega_k(position, params)**2/(G*stokes_peb))


# print(M_3D_2DH_trans(sim.position[:,0].value, sim_params.t_in, params))
fig, axs =plt.subplots(1,1, figsize = (8,8))
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_drift_ep(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st drift epstein', linestyle = '--')
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_drift_st(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st drift stokes', linestyle = '--')
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans_debug_frag(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'st fragt', linestyle = '--') 
axs.loglog(sim_irr_10.position[:,0].to(u.au), M_3D_2DH_trans(sim_irr_10.position[:,0].value, sim_params_irr_10.t_fin, params), label = 'stokes ', color = 'black')

axs.set_ylim(1e-6, 1e2)
axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_ylabel('$M_{\mathrm{3D/2D}}$ [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plot.all_y_ticks(axs, 100)
axs.legend()
plt.savefig("figures/transition_mass.png", bbox_inches='tight')